# Section 5.6.1
# Animate decline of head (water table) in a strip

Use the solution for the decline of head in a strip of land within parallel ditches to animat it.

Animation is done with `matplotlib.anamation.FuncAnimation`. This requires two functions. One that initializes the lines to be drawn and the other to update its data in an iteration loop. The latter function takes a single integer input that is the frame number within the iteration.

It makes sense to define a class, in this case "Strip" which has a method to compute itself. Doing this, the class is instantiated with all its data before running the animation, so that the aninamtion only requires the counter.

@TO 2020-12-09

### Strip formula

$$s(x, t) = 2 h_0 \sum_{i=0}^\infty \left\{ \frac {(-1)^i} {(i + \frac 1 2) \pi} \cos \left[(i + \frac 1 2) \pi \frac x b \right]
\exp \left[-(i + \frac 1 2 )^2 \pi^2 \frac t T \right]
\right\},\,\,\,\,T=\frac{b^2 S}{kD}$$

In [153]:
attribs = lambda obj: [o for o in dir(obj) if not o.startswith('_')]

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import pdb


def newfig(title='title', xlabel='xlabel', ylabel='ylabel',
                   xlim=None, ylim=None, size_inches=(8, 6)):
    fig, ax = plt.subplots()
    fig.set_size_inches(size_inches)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if xlim:
        ax.set_xlim(xlim)
    if ylim:
        ax.set_ylim(ylim)
    ax.grid()
    return fig, ax

In [154]:
# First set up the figure, the axis, and the plot element we want to animate
class Strip:
    
    def __init__(self, b=None, h0=None, kD=None, S=None, n=20):
        """Return a strip object.
        
        Parameters
        ----------
        b: float
            half-width of the strip
        h0: float
            initial head above drainage base
        kD: float
            transmissivity
        S: float
            Storage coefficient
        n: number of points in the half-strip
        """
        self.b = b
        self.h0 = h0
        self.kD = kD
        self.S = S
        self.n = n
        self.x = np.linspace(-b, b, n + 1)
        self.T = self.b * 2 * self.S / self.kD
        self.T50 = (2 / np.pi) ** 2 * np.log(2) * self.T
        
    def compute(self, t, n=20):
        s = np.zeros_like(self.x)
        for i in range(n):
            pif = (i + 0.5) * np.pi
            s += (-1) ** i / pif * np.cos(pif * self.x / self.b) \
                            * np.exp(- pif ** 2 * t / self.T)
        return 2 * self.h0 * s
    
    def plot(self, times=None, n=20, ax=None):
        """Plot the head for all times."""
        for t in times:
            s = self.compute(t)
            ax.plot(self.x, s, label='t= {:.3f} d'.format(t))
    

In [162]:
data = {'kD': 600, 'S':0.2, "h0":2, "b":75, "n":20}

strip = Strip(**data)

title = "Animation of head decline in a land strip.\n" +\
        'kD={:.0f} m2/d, S={:.2f}, b={:.0f} m, h0={:.2f} m'\
        .format(*[data[k] for k in ['kD', 'S', 'b', 'h0']])

fig, ax = newfig(title, "x [m]", "head / water table [m]",
                  xlim=(-data['b'], data['b']), ylim=(0, data['h0']),
                  size_inches=(12, 8))

line, = ax.plot([], [], lw=2)
text  = ax.text(0.05, 0.90, '', fontsize=14, transform=ax.transAxes)
ax.text(0.05, 0.8, 'Water-table decline halftime T50 = {:.3f} d'
                    .format(strip.T50), fontsize=14, transform=ax.transAxes)


n = 6  # total time 6 times the halftime
m = 30 # subdivision of each halftime
t50times = strip.T50 * np.arange(n)
times    = strip.T50 * np.arange(0, n, 1/m)[1:]

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return line, # receiver expects a tuple of the artists involved

# animation function.  This is called sequentially
def animate(t, strip, t50times, ax):
    # global declaration not strictly needed, but nice to make clear
    global line, text
    T50 = strip.T50
    y = strip.compute(t)
    line.set_data(strip.x, y)
    text.set_text('time = {:8.3f} d, t_rel = {:.1f}'.format(t, t/T50))
    
    if t in t50times:
        ax.plot(strip.x, strip.compute(t), label='t / t50 = {:.1f}'.format(t / T50))
    if t == t50times[-1]:
        ax.legend()
    return line, # receiver expects a tuple of the artitsts involved

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=times, fargs=(strip, t50times, ax),
                               interval=100, blit=True)

plt.close(anim._fig)

In [159]:
#HTML(anim.to_html5_video())

In [160]:
# save the animation as an mp4.  This requires ffmpeg or mencoder to be
# installed.  The extra_args ensure that the x264 codec is used, so that
# the video can be embedded in html5.  You may need to adjust this for
# your system: for more information, see
# http://matplotlib.sourceforge.net/api/animation_api.html
anim.save('Head_decline_in_strip.mp4', fps=10, extra_args=['-vcodec', 'libx264'])
print('..finished.')
print(anim.save_count, " frames saved.")

..finished.
179  frames saved.


In [163]:
out = HTML(anim.to_html5_video())
out